<a href="https://colab.research.google.com/github/wtergan/ML_notebooks/blob/main/intro_to_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple notebook for demonstrating and testing different prompts/ prompt techniques.

Based on notebook by DAIR.AI | Elvis Saravia.
  https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-lecture.ipynb

Edited, rewritten, modified from the original for educational purposes.

In [ ]:
# Downloading, installation of libraries, dependencies. Usage of %%capture for suppression of installation outputs.
%%capture
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv

In [ ]:
# Importation of libraries, dependencies.
import openai, os, IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv, find_dotenv

In [ ]:
# Loading of environment variables (.env file can be used for OPENAI_API_KEY).
load_dotenv("/content/OPENAI_API_KEY.env")
# API config.
openai.api_key = os.getenv("OPENAI_API_KEY")

# For langchain.
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

### Prompt basics.

In [ ]:
# Basic functions used for text generation.
"""
Function that sets the parameters needed for text generation.
@param model: (str) Name of the OpenAI model that will be used for text generation.
@param temperature: (int) Parameter used to specify the randomness of the text generation.
@param max_tokens: (int) The maximum number of tokens to generation.
@param top_p: (int) Indicates the probabilty threshold for token sampling. 1 means all tokens are considered.
@param frequency_penalty: (int) Penalty for repeating the same token or phrase in the gneerated text.
@param presence_penalty: (int) Penalty for using tokens that have already been used in the generated text.

@returns openai_params: (dict) Dictionary of the required parameters specified.
"""
def set_open_params(
    model="text-davinci-003",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ Set OpenAI parameters for usage. """
    openai_params={}

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

"""
Function that gets the prompt completion (text generation) from the OpenAI API.
@param params: (dict) Dictionary of the required parameters.
@param prompt: (str) String of text which the model will be responding to.

@returns response: (dict) Dictionary of information of the response, including the
                   'choices' key, which contains the text completion.
"""
def get_completion(params, prompt):
  """ Get completion from the OpenAI API. """
  response = openai.Completion.create(
      engine=params['model'],
      prompt=prompt,
      temperature=params['temperature'],
      max_tokens=params['max_tokens'],
      top_p=params['top_p'],
      frequency_penalty = params['frequency_penalty'],
      presence_penalty = params['presence_penalty'],
  )
  return response

In [ ]:
# Basic prompt example.
params = set_open_params()
prompt = "The sky is"
response = get_completion(params, prompt)
response['choices'][0]['text']

' blue\n\nThe sky is blue because molecules in the atmosphere scatter blue light from the sun more than they scatter red light. This is known as Rayleigh scattering.'

In [ ]:
# We can present the response better (in markdown), by using IPython.
IPython.display.Markdown(response.choices[0]['text'])

 blue

The sky is blue in color due to the way the atmosphere scatters sunlight. Blue is scattered more than other wavelengths of light, so it is seen more often than other colors.

In [ ]:
# We can change the parameters to get slightly different results. Lets attempt to change the temeperature.
params = set_open_params(temperature=0)
prompt = "The sky is"
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 blue

The sky is blue because of the way the atmosphere scatters sunlight. When sunlight passes through the atmosphere, the blue wavelengths are scattered more than the other colors, making the sky appear blue.

### Text Summarization.

Summarization of prompts into quick and easy-to-read summaries. Can always specify to the model how to summarize (for example, summarize as if I am a child, or changing the length of the summary).

In [ ]:
params = set_open_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing
the bacteria or preventing them from reproducting, allowing the body's immune system to fight off the infection.
Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered
intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic
resistance. Explain the above in one sentence:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



Antibiotics are medications used to treat bacterial infections by either killing the bacteria or inhibiting their reproduction, but they are not effective against viral infections and should be used responsibly to avoid developing antibiotic resistance.

Lets instruct the model to explain the paragraph in one sentence like "I am a 5 year old".

In [ ]:
params = set_open_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing
the bacteria or preventing them from reproducting, allowing the body's immune system to fight off the infection.
Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered
intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic
resistance. Explain the above in one sentence, and explain as if I am 5 years old:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



Antibiotics are special medicines that help your body fight off bad bacteria, but they won't work against colds or other virus germs.

### Information Extraction

Retrieval of information from the prompt provided.

In [ ]:
prompt = """Author-contribution statements and acknowledgements in research papers should state clearly and specifically
whether, and to what extent, the authors used AI technologies such as ChatGPT in the preparation of their manuscript and
analysis. They should also indicate which LLMs were used. This will alert editors and reviewers to scrutinize manuscripts
more carefully for potential biases, inaccuracies and improper source crediting. Likewise, scientific journals should
be transparent about their use of LLMs, for example when selecting submitted manuscripts.

Mention the large language model based product mentioned in the paragraph above:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



The large language model mentioned in the paragraph is ChatGPT.

In [ ]:
prompt = """You can play with different variants of 4bit quantization such as NF4 (normalized float 4 (default))
or pure FP4 quantization. Based on theoretical considerations and empirical results from the paper, we recommend using
NF4 quantization for better performance.

Based on the following paragraph, which quantization is better for performance? Only give me a one word answer.
"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)


NF4

### Question Answering

Extension of information extraction; We can ask the model to answer specific questions based on prompt given. Q: A: format to be concise (or any variation of such). Combine instructions, context, input, and output indicators for better response results. Essentially: the better the question, the better the answer.

In [ ]:
prompt = """Answer the question based on the context below. Keep the answer short. Respond "Unsure about answer" if not
sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists
generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind
to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent
organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: What was OKT3 originally sourced from?

Answer:
"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

Mice

Lets edit the prompt so we can get the model to respond that it is not sure about the answer.

In [ ]:
prompt = """Answer the question based on the context below. Keep the answer short. Respond "Unsure about answer" if not
sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists
generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind
to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent
organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: Who was the first patient that recieved a kidney transplant wiht the help of Teplizumab/OKT3?

Answer:
"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

Unsure about answer.

### Text Classification

Categorize/labeling of given text. For example, sentiment analysis. Can make response to prompt better by providing examples for the model.

In [ ]:
prompt = """Classify the text into neutral, negative, or positive.

Text: I think the food was okay.

Sentiment:"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Neutral

In [ ]:
# Providing examples within the prompt for better responses.
# In this instance, we will provide model some examples of text and sentiment pairs,
# with the sentiments in a very specific format.
prompt = """Classify the text into neutral, negative, or positive.

Text: The car show was 3 hours long.
Sentiment: NEUTRaL

Text: I failed the math exam today and it made me very unhappy.
Sentiment: BaDDD!

Text: Going to the beach today was so fun!
Sentiment: GOooD.

Text: I think the food was okay.
Sentiment:"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 NEUTRaL

In [ ]:
# Lets modify the above prompt to instruct the model to give us an explanation to the answer.
prompt = """Classify the text into neutral, negative, or positive. Then, give me an explanation to the answer you
selected (only the sentiment for the last text.)

Text: The car show was 3 hours long.
Sentiment: NEUTRaL

Text: I failed the math exam today and it made me very unhappy.
Sentiment: BaDDD!

Text: Going to the beach today was so fun!
Sentiment: GOooD.

Text: I think the food was okay.
Sentiment:"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 NEUTRAL.

Explanation: The sentiment for the last text is neutral because it does not express a strong opinion either way.

### Role Playing

We can instruct the model on how to behave, respond, and give it the appearance of an identity. This can prove very useful for conversational systems like chatbots. Let the model role play as a 200IQ theoretical physiscist and its response can "simulate" such a role.

In [ ]:
prompt = """The following is a conversation with an AI research assistant. The assistant tone is technical
and scientific.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Sure! A black hole is a region of spacetime exhibiting gravitational acceleration so strong that nothing—no particles or even electromagnetic radiation such as light—can escape from it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole. The boundary of the region from which no escape is possible is called the event horizon.

In [ ]:
# Lets modify the prompt so that the response is concise.
prompt = """The following is a conversation with an AI research assistant. The assistant tone is
technical and scientific. The assistant's responses are to be concise and short.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Sure. A black hole is an object in space created when a very large star collapses in on itself due to its immense gravity. This collapse causes the star to become so dense that not even light can escape its gravitational pull.

### Code Generation

LLMs, unsurprisingly, can output code as well. In fact, it is generally pretty good at it (if the specific model used has been sufficently trained on code data.

In [ ]:
prompt = """Given the following text, I want you to give me the equivalent python code.

/* Ask the user for their name and say "Hello" */
"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)


name = input("Please enter your name: ")
print("Hello " + name)

In [ ]:
prompt = """
Table departments, columns = [DepartmentId, DepartmentName]
Table students, columns = [DepartmentId, StudentId, StudentName]
Create a MySQL query for all students in the Computer Science Department
"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)


SELECT StudentId, StudentName 
FROM students 
WHERE DepartmentId = (SELECT DepartmentId 
                      FROM departments 
                      WHERE DepartmentName = 'Computer Science');

### Reasoning

Very difficult for current LLMs to achieve currently (though possible, like in GPT-4, which shows some advanced reasoning skills). Such examples of reasoning includes solving mathemetical problems, puzzles, etc. Often needs advanced prompting techniques like Chain of Thought or self-consistency for better results.

In [ ]:
# Lets start with a simple math problem that the model should solve.
prompt = """ What is 10000 * 10000?
"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)


100000000

Lets now ask the model to solve a more difficult task (Expected answer is 41).

In [ ]:
prompt = """Give me the sum of the numbers in this group (whose sum will be an even number): 15, 32, 5, 13, 82, 7, 1.
A: """
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 135

The above cell outputs an incorrect answer. Lets modify the prompt so that it can figure out the right answer.

In [ ]:
prompt = """Give me the sum of the numbers in this group (whose sum will be an even number): 15, 32, 5, 13, 82, 7, 1.
Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



Odd numbers: 15, 5, 13, 7, 1
Sum of odd numbers = 41 (odd)

Now, add the even numbers:

Even numbers: 32, 82
Sum of even numbers = 114 (even)

Finally, add the sum of the odd numbers and the sum of the even numbers together:

41 + 114 = 155 (odd)

In [ ]:
# Lets improve this prompt even further for better results.
prompt = """Give me the sum of the numbers in this group (whose sum will be an even number): 15, 32, 5, 13, 82, 7, 1.
Solve by breaking the problem into steps:
Step 1. identify the odd numbers. This can be achieved by determining if each number is divisible by 2 evenly.
If not, then it is a odd number.
Step 2. After determining the odd numbers in the group, add all of them together.
Step 3. Determine whether the sum of the odd numbers is odd or even. The sum of them should be odd. If not, then
the answer is not correect.
Complete each step and then give me your answer.
"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

Answer: The sum of the odd numbers in the group is 27 (15 + 5 + 13 + 7 + 1). The sum is an odd number.

Notice how the answer is still not correct. In this case, we can either use more advanced prompting techniques (which we are about to get into) in order to extract the full capabilities of the model itself, or alternatively, finetune the model on data that better elucidates reasoning (such as a math dataset), so that the model can answer the question more precisely, since in some cases prompting will simply not be enough.

### Few-Shot Prompting

We have seen this before: prompting the model with some examples in order for the model to make better responses to the prompt. To expand on this, when one provides examples for the model, it enables in-context learning, in which the model is steered for better performance of the task based on the demonstrations provided.

This is in contrast to zero-shot prompting, which does not involve and examples within the prompt.

In [ ]:
# Basic few-shot example. Notice the lack on consistency in the format of the prompt.
prompt = """
Positive This is awesome!
This is bad! Negative
Wow that movie was rad!
Positive
What a horrible show! --
"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

Negative

While it is better than zero-shot prompting at some tasks, it doesn't do as good a job for tasks that requires reasoning. For example:

In [ ]:
# Few shot example for more complex tasks.
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 The answer is True.

As you can see, the answer (which is supposed to be False, 41), gives us True, which is incorrect. So we can create more sophisticated prompting techiques for better resilts.

Chain of Thought Prompting (CoT)

Enables better reasoning capabilities through intermediate reasoning steps.

In [ ]:
# Example One-shot CoT prompt.
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.

The response in the above example is correct. This proves better than the few-shot example earilier where it returned True.

Now, lets fuse CoT with few shot prompting.

In [ ]:
# CoT + Few-shot.
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.

### Zero-Shot CoT

Addition of "Let's think step by step" to the prompt.

In [ ]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the
repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



I bought 10 apples: 10 apples 
I gave 2 apples to the neighbor and 2 to the repairman: 8 apples 
I bought 5 more apples: 13 apples 
I ate 1 apple: 12 apples 

Therefore, I remained with 12 apples.

### Self-Consistency

Sampling of multiple reasoning paths via few-shot CoT. Selects the most consistent answer by marginalizing out the sampled reasoning paths.

In [ ]:
prompt = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.

Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.

Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5

Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:"""

outputs = []
for i in range(10):
  response = get_completion(params, prompt)
  outputs.append(response)
for i in range(len(outputs)):
  print("Output", i,":", outputs[i].choices[0].text)


Output 0 :  When I was 6, my sister was half my age. That means she was 6/2 = 3 years old.
Now I'm 70, so my sister must be 70/2 = 35 years old. The answer is 35.
Output 1 :  When my sister was 6, I was 6. Now I'm 70, so my sister must be 70 / 2 = 35 years old. The answer is 35.
Output 2 :  When you were 6, your sister was half your age. This means she was 6/2 = 3. Now that you are 70, your
sister is 70/2 = 35. The answer is 35.
Output 3 :  The sister was half the age of the person when he was 6 years old. So, when he was 6, his sister was 3.

Now he is 70, so his sister is 70 - 3 = 67 years old. The answer is 67.
Output 4 :  When I was 6 my sister was half my age, so she was 3. Now I am 70, so she is 70 - 3 = 67. The answer is 67.
Output 5 :  When you were 6, your sister was 3. Now you are 70, so your sister must be 70/2 = 35. The answer is 35.
Output 6 :  When you were 6, your sister was half your age, so she was 3. Now you are 70, so your sister is 70/2 = 35. The answer is 35.
Outpu

Self-consistency generates multiple reasoning answering pairs and aggregates via a majority vote to find the most conistent answer. We can either take the unweighted sum/majority vote over the answer (which answer is more frequent), or we can compute the unnormalized log probabilities of the output and then use the softmax to normalize them. We can then take the weighted sum as the weights on each answer. Lets demonstrate this:

In [ ]:
# Lets first show the method of simply taking the unweighted sum/ majortiy vote over the answer.
import numpy as np

answers = []
# Extracting the exact numerical answer from each of the outputs.
for i in range(len(outputs)):
  answers.append(int(outputs[i].choices[0].text[-3:].replace(".", "")))
print(f"Numerical answers from output: {answers}")

unique, counts = np.unique(answers, return_counts=True)
index = np.argmax(counts)
print(f"Best answer: {unique[index]}")

ValueError: ignored

### Generated Knowledge Prompting

Generating knowledge from a language model, then providing the knowledge as additional input when answering a question.


In [ ]:
prompt = """Part of golf is trying to get a higher point total than the others. Yes or no?"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

In [ ]:
prompt = """Input: Greece is larger than mexico.
Knowledge: Greece is approximately 131,957 sq km, while Mexico is approximately 1,964,375 sq km, making Mexico 1,389% larger than Greece.

Input: Glasses always fog up.
Knowledge: Condensation occurs on eyeglass lenses when water vapor from your sweat, breath, and ambient humidity lands on a cold surface, cools, and then changes into tiny drops of liquid, forming a film that you see as fog. Your lenses will be relatively cool compared to your breath, especially when the outside air is cold.

Input: A fish is capable of thinking.
Knowledge: Fish are more intelligent than they appear. In many areas, such as memory, their cognitive powers match or exceed those of ’higher’ vertebrates including non-human primates. Fish’s long-term memories help them keep track of complex social relationships.

Input: A common effect of smoking lots of cigarettes in one’s lifetime is a higher than normal chance of getting lung cancer.
Knowledge: Those who consistently averaged less than one cigarette per day over their lifetime had nine times the risk of dying from lung cancer than never smokers. Among people who smoked between one and 10 cigarettes per day, the risk of dying from lung cancer was nearly 12 times higher than that of never smokers.

Input: A rock is the same size as a pebble.
Knowledge: A pebble is a clast of rock with a particle size of 4 to 64 millimetres based on the Udden-Wentworth scale of sedimentology. Pebbles are generally considered larger than granules (2 to 4 millimetres diameter) and smaller than cobbles (64 to 256 millimetres diameter).

Input: Part of golf is trying to get a higher point total than others.
Knowledge:"""

### Generating Data

Self-explanatory: make the LLM generate data via proper prompting.

In [ ]:
prompt = """Produce ONLY 10 exemplars for sentiment analysis. Examples are categorized as either positive or negative. Produce 2 negative
examples and 8 positive examples. Use this format for the examples:

Q: <sentence>
A: <sentiment>"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



Q: I just got a promotion at work!
A: Positive

Q: I'm so grateful for the help my friends and family gave me.
A: Positive

Q: I'm so disappointed that I failed the test.
A: Negative

Q: I've been so stressed lately.
A: Negative

Q: I'm so proud of the results I achieved.
A: Positive

Q: I'm so excited for the upcoming vacation.
A: Positive

Q: I can't believe how hard this project is.
A: Negative

Q: I'm so happy to have experienced this journey.
A: Positive

Q: I feel so overwhelmed with all this work.
A: Negative

Q: I'm so fortunate to have such great colleagues.
A: Positive

Q: I'm so discouraged by the lack of progress.
A: Negative

Q: I'm so content with all that I have accomplished.
A: Positive

In [ ]:
prompt = """produce 3 wine reviews and label taste, flavor, aroma related token; present the result as a json file
, in addition add the coordinate of each term for NER task"""
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



{
  "review_1": {
    "text": "This is a full-bodied, earthy red wine with notes of blackberry and oak. The tannins are well-balanced and the finish is robust.",
    "tokens": [
      {
        "token": "full-bodied",
        "taste": true,
        "flavor": false,
        "aroma": false,
        "coord": [0, 11]
      },
      {
        "token": "earthy",
        "taste": false,
        "flavor": true,
        "aroma": false,
        "coord": [12, 18]
      },
      {
        "token": "blackberry",
        "taste": false,
        "flavor": true,
        "aroma": true,
        "coord": [30, 39]
      },
      {
        "token": "oak",
        "taste": false,
        "flavor": true,
        "aroma": true,
        "coord": [40, 43]
      },
     

### PAL (Program-Aided Language Models): Code as Reasoning

Uses the model to read natural language problems and generate programs as the intermediate reasoning steps, but offloads the solution step to a runtime such as Python Interpreter. This is in contrast to advanced prompting techniques such as Chain of Thought, which produces the intermediate steps via free-form text.

Lets develop an application that is able to take in some data and answer questions about the data input, ie. reason about the question being asked through code.

In [ ]:
# Creation of an instance of the text-davinci-003 model.
llm = OpenAI(model_name="text-davinci-003", temperature=0.7)

In [ ]:
# Question in which to ask the model.
question = "Which is the oldest penquin?"

The following cell consists of the prompt in which to give the model. It contains examples of prompts and its subsequent reasoning steps, which are in the form of code.

For example, the first example prompt (which we will call sub-prompts) shows a few penquins' name, age, height, and weight. It then asks "How many penquins are less than 8 years old"? This is followed by the reasoning steps, which puts the penguins in a python list, adds a fourth penguin, and then find the number of penguins less than 8 years old.

The intuition is that the model will follow the logic of example code, and when given a new fresh prompt, it will follow the same reasoning steps to find the novel answer.

In [ ]:
prompt = '''
"""
Q: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg)
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm.
We now add a penguin to the table:
James, 12, 90, 12
How many penguins are less than 8 years old?
"""
# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Add penguin James.
penguins.append(('James', 12, 90, 12))
# Find penguins under 8 years old.
penguins_under_8_years_old = [penguin for penguin in penguins if penguin[1] < 8]
# Count number of penguins under 8.
num_penguin_under_8 = len(penguins_under_8_years_old)
answer = num_penguin_under_8
"""
Q: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg)
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm.
Which is the youngest penguin?
"""
# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort the penguins by age.
penguins = sorted(penguins, key=lambda x: x[1])
# Get the youngest penguin's name.
youngest_penguin_name = penguins[0][0]
answer = youngest_penguin_name
"""
Q: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg)
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm.
What is the name of the second penguin sorted by alphabetic order?
"""
# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort penguins by alphabetic order.
penguins_alphabetic = sorted(penguins, key=lambda x: x[0])
# Get the second penguin sorted by alphabetic order.
second_penguin_name = penguins_alphabetic[1][0]
answer = second_penguin_name
"""
{question}
"""
'''.strip() + "\n"

In [ ]:
llm_output = llm(prompt.format(question=question))
print(llm_output)

# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort the penguins by age.
penguins = sorted(penguins, key=lambda x: x[1], reverse=True)
# Get the oldest penguin's name.
oldest_penguin_name = penguins[0][0]
answer = oldest_penguin_name


In [ ]:
exec(llm_output)
print(answer)

Vincent


Lets now try a different question and see the results.

In [ ]:
question = "Which is the second youngest penquin?"
llm_output = llm(prompt.format(question=question))
print(llm_output)

exec(llm_output)
print(answer)

# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort the penguins by age.
penguins = sorted(penguins, key=lambda x: x[1])
# Get the second youngest penguin's name.
second_youngest_penguin_name = penguins[1][0]
answer = second_youngest_penguin_name
Louis


Louis is indeed the second youngest penquin's name.

Lets try another example:



In [ ]:
llm = OpenAI(model_name='text-davinci-003', temperature=0)

In [ ]:
question = "Today is 27 Feburary 2023. I was born exactly 25 years ago. What is the date I was born in MM/DD/YYYY?"

In [ ]:
DATE_UNDERSTANDING_PROMPT = """
# Q: 2015 is coming in 36 hours. What is the date one week from today in MM/DD/YYYY?
# If 2015 is coming in 36 hours, then today is 36 hours before.
today = datetime(2015, 1, 1) - relativedelta(hours=36)
# One week from today,
one_week_from_today = today + relativedelta(weeks=1)
# The answer formatted with %m/%d/%Y is
one_week_from_today.strftime('%m/%d/%Y')
# Q: The first day of 2019 is a Tuesday, and today is the first Monday of 2019. What is the date today in MM/DD/YYYY?
# If the first day of 2019 is a Tuesday, and today is the first Monday of 2019, then today is 6 days later.
today = datetime(2019, 1, 1) + relativedelta(days=6)
# The answer formatted with %m/%d/%Y is
today.strftime('%m/%d/%Y')
# Q: The concert was scheduled to be on 06/01/1943, but was delayed by one day to today. What is the date 10 days ago in MM/DD/YYYY?
# If the concert was scheduled to be on 06/01/1943, but was delayed by one day to today, then today is one day later.
today = datetime(1943, 6, 1) + relativedelta(days=1)
# 10 days ago,
ten_days_ago = today - relativedelta(days=10)
# The answer formatted with %m/%d/%Y is
ten_days_ago.strftime('%m/%d/%Y')
# Q: It is 4/19/1969 today. What is the date 24 hours later in MM/DD/YYYY?
# It is 4/19/1969 today.
today = datetime(1969, 4, 19)
# 24 hours later,
later = today + relativedelta(hours=24)
# The answer formatted with %m/%d/%Y is
today.strftime('%m/%d/%Y')
# Q: Jane thought today is 3/11/2002, but today is in fact Mar 12, which is 1 day later. What is the date 24 hours later in MM/DD/YYYY?
# If Jane thought today is 3/11/2002, but today is in fact Mar 12, then today is 3/1/2002.
today = datetime(2002, 3, 12)
# 24 hours later,
later = today + relativedelta(hours=24)
# The answer formatted with %m/%d/%Y is
later.strftime('%m/%d/%Y')
# Q: Jane was born on the last day of Feburary in 2001. Today is her 16-year-old birthday. What is the date yesterday in MM/DD/YYYY?
# If Jane was born on the last day of Feburary in 2001 and today is her 16-year-old birthday, then today is 16 years later.
today = datetime(2001, 2, 28) + relativedelta(years=16)
# Yesterday,
yesterday = today - relativedelta(days=1)
# The answer formatted with %m/%d/%Y is
yesterday.strftime('%m/%d/%Y')
# Q: {question}
""".strip() + '\n'

In [ ]:
llm_output = llm(DATE_UNDERSTANDING_PROMPT.format(question=question))
print(llm_output)

# If today is 27 Feburary 2023 and I was born exactly 25 years ago, then I was born 25 years before.
today = datetime(2023, 2, 27)
# I was born,
born = today - relativedelta(years=25)
# The answer formatted with %m/%d/%Y is
born.strftime('%m/%d/%Y')


In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
exec(llm_output)
print(born)

1998-02-27 00:00:00
